In [2]:
# 從判決書資料庫抓取資料
import pymysql
import pandas as pd
def Get_Data_From_Mysql(table_name,keyword):
    contents_list=[]
    target_ids =[]
    # 自行填入 host & passwd ， 在此處不提供
    db = pymysql.connect(host="YOUR HOST",port=3306, user="public_author",passwd="YOUR PASSWORD",db="AML_News" ,charset='utf8')
    try:
        with db.cursor() as cursor:
            if(keyword=="all"):
                sql0 = "SELECT * FROM `"+table_name+"`"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
            else: 
                sql0 = "SELECT * FROM `"+table_name+"` WHERE '"+keyword+"' LIKE CONCAT('%', `keyword`)"
                cursor.execute(sql0)
                contents_list = cursor.fetchall()
                
        db.commit()
    finally:
        db.close()
    return  pd.DataFrame(list(contents_list), columns=['id','name','article','keyword'])

In [3]:
# 自行填入 host & passwd ， 在此處不提供
# 此為本組客製化之資料庫
import pymysql
import pandas as pd

def Sent_Data_To_Mysql(name, article, keyword, table_name, date, location):
    db = pymysql.connect(host="YOUR HOST",port=3306, user="public_author",passwd="YOUR PASSWORD",db="AML_News" ,charset='utf8')
    content = db.escape_string(article)
    try:
        with db.cursor() as cursor:
            sql0 = "SELECT `id` FROM `"+table_name+"` WHERE '"+article+"' LIKE CONCAT('%', `article`, '%') AND '"+ location +"' LIKE CONCAT('%', `location`, '%') AND '"+ name +"' LIKE CONCAT('%', `name`, '%') "
            cursor.execute(sql0)
            duplicate_id = cursor.fetchone()
            if(duplicate_id is not None):#如果已經存在一個content重複的
                sql1 = " INSERT INTO `"+table_name+"` (`id`, `name`,`article`,`keyword`,`location`,`date`) VALUES (%s,%s,%s,%s,%s,%s) "
                sql2 = " ON DUPLICATE KEY UPDATE `article`= VALUES(`article`),`name`= VALUES(`name`),`keyword`= VALUES(`keyword`),`location`= VALUES(`location`),`date`= VALUES(`date`)"
                cursor.execute(sql1+sql2, (str(duplicate_id[0]), article,name,keyword,location,date))
                print(str(duplicate_id[0]))
            else:#如果沒有重複的
                sql = "INSERT INTO `"+table_name+"`(`article`,`name`,`keyword`,`location`,`date`) VALUES ( %s,%s,%s,%s,%s)"
                cursor.execute(sql, (article,name,keyword,location,date))
        db.commit()
    finally:
        db.close()

#content_type 可以是 keyword, location, date 或是 name 
def Get_Data_From_Mysql_by_content(table_name,content,content_type):
    contents_list=[]
    target_ids =[]
    db = pymysql.connect(host="YOUR HOST",port=3306, user="public_author",passwd="YOUR PASSWORD",db="AML_News" ,charset='utf8')
    try:
        with db.cursor() as cursor:
            if(content_type is 'date'):
                sql0 = "SELECT * FROM `"+table_name+"` WHERE `"+content_type+"` LIKE '"+content+"'"
            else:
                sql0 = "SELECT * FROM `"+table_name+"` WHERE `"+content_type+"` LIKE CONCAT('%', '"+content+"','%')"
            cursor.execute(sql0)
            contents_list = cursor.fetchall()
                
        db.commit()
    finally:
        db.close()
    return  pd.DataFrame(list(contents_list), columns=['id','name','article','keyword','date','location'])

In [5]:
def Get_Address(context):
    address = ""
    index = context.index('灣')
    address = context[index+1 ] + context[index+2 ]
#     print(address)
    return address
def Get_Year(context):
    year = ""
    index = context.index('年')
    for x in range(0,index):
        if (context[x].isdigit()):
            year+=context[x]
#     print(year)
    return year

In [7]:
# 從判決書資料庫抓取資料
# 並對判決書內文進行抓取 地點 和 年份
# 抓取後丟入丟組資料庫中

from datetime import datetime
table_name = 'judical_criminal_people'

keyword='洗錢'

output_dataframe = Get_Data_From_Mysql(table_name,keyword)

for index,row in output_dataframe.iterrows():
    name = row[1].lstrip()
    article = row[2]
    keyword = row[3]
    table_name = 'group_11'
    
    location = Get_Address(row[2])
    date_Y = int(Get_Year(row[2])) + 1911
    
    date =  str(date_Y) + '年1月1'    
    date = datetime.strptime(date, "%Y年%m月%d")#注意是大寫的Y 換成小寫會出錯
    date = date.strftime('%Y-%m-%d') 
    
    print(name)
#     print(article)
    print(keyword)
    print(date)
    print(location)
    
    #Sent_Data_To_Mysql(name, article, keyword, table_name, date, location)


李慧慧
洗錢
2019-01-01
新竹
王菏勵
洗錢
2018-01-01
新竹
余恩中
洗錢
2019-01-01
苗栗
楊光正
洗錢
2019-01-01
新竹
黃彥銘
洗錢
2018-01-01
新竹
彭功明
洗錢
2018-01-01
苗栗
林緯隆
洗錢
2018-01-01
苗栗
楊凱翔
洗錢
2019-01-01
基隆
程彥傑
洗錢
2019-01-01
新竹
鄭名含
洗錢
2019-01-01
新竹
劉振佐
洗錢
2019-01-01
新竹
謝汶琪
洗錢
2019-01-01
基隆
文樊聖
洗錢
2018-01-01
基隆
劉宗玄
洗錢
2019-01-01
新竹
范家瑜
洗錢
2019-01-01
新竹
撒扶烙‧巴雅斯
洗錢
2018-01-01
新竹
傅宗明
洗錢
2019-01-01
苗栗
傅宗明
洗錢
2018-01-01
苗栗
梁騰方
洗錢
2019-01-01
新竹
周俊誠
洗錢
2019-01-01
新竹
劉建良
洗錢
2019-01-01
基隆
楊雅琁
洗錢
2019-01-01
基隆
林柏佑
洗錢
2019-01-01
苗栗
曹月美
洗錢
2019-01-01
基隆
古民雄
洗錢
2018-01-01
新竹
陳博均
洗錢
2018-01-01
新竹
許國陞　
洗錢
2018-01-01
基隆
陳盈辰
洗錢
2019-01-01
彰化
范振軒
洗錢
2019-01-01
新竹
江清琮
洗錢
2018-01-01
新竹
鄭弘翌
洗錢
2019-01-01
新竹
謝武志
洗錢
2019-01-01
基隆
李政洋
洗錢
2019-01-01
新竹
陳彥伶
洗錢
2019-01-01
基隆
范勝閎
洗錢
2019-01-01
新竹
謝函書
洗錢
2018-01-01
新竹
陳建廷
洗錢
2018-01-01
苗栗
林億雯
洗錢
2019-01-01
基隆
馬傑
洗錢
2019-01-01
基隆
李珮綺
洗錢
2019-01-01
基隆
郭育維
洗錢
2019-01-01
基隆
黃展昱
洗錢
2019-01-01
新竹
李政洋
洗錢
2019-01-01
新竹
郭淑瓊
洗錢
2019-01-01
新竹
洪春平　(大陸地區人士)
洗錢
2019-01-01
新竹
李俊義　
洗錢
2019-01-01
橋頭
蘇峻賢
洗錢
2019-01-01
基隆

In [8]:
table_name = 'group_11' #這個不可以改變，為你們設定的
content = '洗錢' #我用模糊比對來寫，因此只要有出線的詞就會被選中
content_type = 'keyword'
Get_Data_From_Mysql_by_content(table_name,content,content_type)

,id,name,article,keyword,date,location
0,2,臺灣新竹地方法院刑事簡易判決 108年度金簡字第21號\n聲 請 人 臺灣新竹地方檢...,新竹,洗錢,2019-01-01,李慧慧
1,3,臺灣新竹地方法院刑事簡易判決 107年度金簡字第30號\n聲 請 人 臺灣新竹地方檢...,新竹,洗錢,2018-01-01,王菏勵
2,4,臺灣苗栗地方法院刑事裁定 108年度金訴字第8號\n公 訴 人 臺灣苗栗地方檢察...,苗栗,洗錢,2019-01-01,余恩中
3,5,臺灣新竹地方法院刑事簡易判決 108年度竹簡字第129號\n聲 請 人 臺灣新竹地方檢...,新竹,洗錢,2019-01-01,楊光正
4,6,臺灣新竹地方法院刑事判決 107年度訴字第868號\n公 訴 人 臺灣新竹地方檢...,新竹,洗錢,2018-01-01,黃彥銘
5,7,臺灣苗栗地方法院刑事判決 107年度金訴字第1號\n公 訴 人 臺灣苗栗地方檢察...,苗栗,洗錢,2018-01-01,彭功明
6,8,臺灣苗栗地方法院刑事判決 107年度原訴字第33號\n公 訴 人 臺灣苗栗地方檢...,苗栗,洗錢,2018-01-01,林緯隆
7,9,臺灣基隆地方法院刑事判決 108年度金簡上字第1號\n上 訴 人 臺灣基隆地方檢察...,基隆,洗錢,2019-01-01,楊凱翔
8,10,臺灣新竹地方法院刑事簡易判決\n 108年度竹東簡字第58...,新竹,洗錢,2019-01-01,程彥傑
9,11,臺灣新竹地方法院刑事簡易判決\n 108年度竹北簡字第144...,新竹,洗錢,2019-01-01,鄭名含
